In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
import os

#### Web Page Elements 
Videos_List : style-scope ytd-rich-item-renderer

title --  //*[@id="video-title-link"]

views -- //*[@id="metadata-line"]/span[1]

time -- //*[@id="metadata-line"]/span[2]


## Scrolling Till 50 Links are reached


In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, 1000);")
    time.sleep(1)

def fetch_video_details(url, max_videos=50):
    PATH = r'chromedriver.exe'
    service = Service(executable_path=PATH)
    wd = webdriver.Chrome(service=service)
    wd.get(url)
    
    video_details = []
    while len(video_details) < max_videos:
        videos = wd.find_elements(By.CLASS_NAME, "style-scope ytd-rich-item-renderer")
        scroll_to_end(wd)
        
        for video in videos:
            title = video.find_element(By.XPATH, ".//*[@id='video-title-link']").text
            view = video.find_element(By.XPATH, ".//*[@id='metadata-line']/span[1]").text
            date = video.find_element(By.XPATH, ".//*[@id='metadata-line']/span[2]").text
            
            details = {
                'title': title,
                'views': view,
                'date': date
            }
            
            video_details.append(details)
            if len(video_details) >= max_videos:
                break
    
    wd.quit()
    return video_details

url = "https://www.youtube.com/@krishnaik06/videos"
max_videos = 50
video_details = fetch_video_details(url, max_videos)
print(video_details)


[{'title': 'Evidently AI Tutorial-Open Source ML Models Monitoring and Observability', 'views': '5.4K views', 'date': '2 days ago'}, {'title': 'Is Transition From Mechanical Engineering To Data Science Possible? Shivam Transition Story Podcast', 'views': '11K views', 'date': '5 days ago'}, {'title': 'BentoML Tutorial: Build Production Grade AI Applications', 'views': '8.2K views', 'date': '7 days ago'}, {'title': 'Rohan Successful Transition-Freshers To MLOPS Lead With Equity @PWSkillsTech #datascience #ai', 'views': '8.2K views', 'date': '9 days ago'}, {'title': 'Projjal Transition Story To Data Science With 380% Hike', 'views': '12K views', 'date': '11 days ago'}, {'title': 'Tanisha Transition Story To Data Analytics Industry With 200% Hike', 'views': '25K views', 'date': '2 weeks ago'}, {'title': 'Tavishi Transition Story To Data Science With 6 Offers And 170% Hike @PWSkillsTech', 'views': '63K views', 'date': '3 weeks ago'}, {'title': 'End To End MLOPS Data Science Project Implemen

In [37]:
pd.DataFrame(video_details)



,title,views,date
0,Evidently AI Tutorial-Open Source ML Models Mo...,5.4K views,2 days ago
1,Is Transition From Mechanical Engineering To D...,11K views,5 days ago
2,BentoML Tutorial: Build Production Grade AI Ap...,8.2K views,7 days ago
3,Rohan Successful Transition-Freshers To MLOPS ...,8.2K views,9 days ago
4,Projjal Transition Story To Data Science With ...,12K views,11 days ago
5,Tanisha Transition Story To Data Analytics Ind...,25K views,2 weeks ago
6,Tavishi Transition Story To Data Science With ...,63K views,3 weeks ago
7,End To End MLOPS Data Science Project Implemen...,18K views,3 weeks ago
8,Data Versioning Hands On Tutorials With DVC Fo...,7.9K views,4 weeks ago
9,The Power Of MLFLOW And Dagshub-Open Source Da...,13K views,1 month ago
